In [ ]:
import pprint
import os
from rdflib import Namespace
from cmmn3 import parseModel, convertModel, convertLog, convertState, reasonAll, showCase, aggregateErrors, showErrors

n3Path = "cmmn3/n3/"
xmlNs = {"cmmn": "http://www.omg.org/spec/CMMN/20151109/MODEL"}
modelNs = Namespace("http://ontario.org/qbp/cap#")

xmlPath = "/Users/wvw/Dropbox/research/projects/Montfort PM/cmmn/cap-all.cmmn"
modelPath = "ttl/cap/model.ttl"
csvFolder = "/Users/wvw/git/pm/mft/data/subsets/"
evtPath = os.path.join(csvFolder, "cap-events-labreq_map.csv")
# statStatePath = os.path.join(csvFolder, "cap-status-admit.csv")
dynStatePath = os.path.join(csvFolder, "cap-clinical_data.csv")
obsPath = "ttl/cap/obs/obs_all.ttl"
statePath = "ttl/cap/state"
imgPath = "js/diagram.png"

# Convert TTL

## Model

In [2]:
itemObjs = parseModel(xmlPath, xmlNs)
# pprint.pprint(itemObjs)
convertModel(itemObjs, modelNs, modelPath)

## Log

In [3]:
convertLog(evtPath, modelNs, destPath=obsPath)

## State

In [ ]:
convertState(evtPath, dynStatePath, modelNs, destPath=statePath)

/Users/wvw/git/pm/decl_cig/cmmn3/cmmn3/convert.py:47: DtypeWarning: Columns (34,35,36,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  value = row['value']


case #0 / 3400
case #100 / 3400
case #200 / 3400
case #300 / 3400
case #400 / 3400


# Check compliance

## Reason

In [ ]:
out = reasonAll(modelPath, obsPath, n3Path, printerr=False)
errors, finals = out
print("errors:", errors.columns)
print("finals:", finals.columns)

## Show

In [ ]:
itemObjs = parseModel(xmlPath, xmlNs)

In [ ]:
showCase(out, '100057', itemObjs, xmlPath, imgPath)

In [ ]:
agg_errors = aggregateErrors(out)
agg_errors

In [ ]:
errors[errors['item']=='oxygen saturation']

In [ ]:
# TODO inactiveToCompleted: show on outer stage

showErrors(agg_errors, itemObjs, xmlPath, imgPath)

In [ ]:
from IPython.display import Image, display

display(Image(filename=imgPath))